In [ ]:
!pip install deepagents
!pip install slack-sdk
!pip install langchain-daytona

In [ ]:
!pip install langchain-daytona

In [37]:
import os
from dotenv import load_dotenv

# Load the environment variables from .env file
load_dotenv()

# Now you can access the environment variable just like before
ANTHROPIC_API_KEY = os.environ.get('ANTHROPIC_API_KEY')
LANGSMITH_API_KEY = os.environ.get('LANGSMITH_API_KEY')
LANGSMITH_TRACING="true"
GOOGLE_API_KEY = os.environ.get('GOOGLE_API_KEY')
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
DAYTONA_API_KEY = os.environ.get('DAYTONA_API_KEY')
SLACK_USER_TOKEN = os.environ.get('SLACK_USER_TOKEN')
SLACK_BOT_TOKEN = os.environ.get('SLACK_BOT_TOKEN')
CHANNEL_ID = os.environ.get('CHANNEL_ID')

In [10]:
from daytona import Daytona

from langchain_daytona import DaytonaSandbox

sandbox = Daytona().create()
backend = DaytonaSandbox(sandbox=sandbox)

In [ ]:
result = backend.execute("echo ready")
print(result)

ExecuteResponse(output='ready', exit_code=0, truncated=False)


In [33]:
import csv
import io

# Create sample sales data
data = [
    ["Date", "Product", "Units Sold", "Revenue"],
    ["2025-08-01", "Widget A", 10, 250],
    ["2025-08-02", "Widget B", 5, 125],
    ["2025-08-03", "Widget A", 7, 175],
    ["2025-08-04", "Widget C", 3, 90],
    ["2025-08-05", "Widget B", 8, 200],
]

# Convert to CSV bytes
text_buf = io.StringIO()
writer = csv.writer(text_buf)
writer.writerows(data)
csv_bytes = text_buf.getvalue().encode("utf-8")
text_buf.close()

# Upload to backend
backend.upload_files([("/home/daytona/data/sales_data.csv", csv_bytes)])

[FileUploadResponse(path='/home/daytona/data/sales_data.csv', error=None)]

In [ ]:
from langchain.tools import tool
from slack_sdk import WebClient


slack_client = WebClient(token=SLACK_BOT_TOKEN)


@tool(parse_docstring=True)
def slack_send_message(text: str, file_path: str | None = None) -> str:
    """Send message, optionally including attachments such as images.

    Args:
        text: (str) text content of the message
        file_path: (str) file path of attachment in the filesystem.
    """
    if not file_path:
        slack_client.chat_postMessage(channel=channel, text=text)
    else:
        fp = backend.download_files([file_path])
        slack_client.files_upload_v2(
            channel=CHANNEL_ID,  
            content=fp[0].content,
            initial_comment=text,
        )

    return "Message sent."

In [35]:
import uuid

from langgraph.checkpoint.memory import InMemorySaver
from deepagents import create_deep_agent


checkpointer = InMemorySaver()

agent = create_deep_agent(
    model="anthropic:claude-sonnet-4-5",
    tools=[slack_send_message],
    backend=backend,
    checkpointer=checkpointer,
)

thread_id = str(uuid.uuid4())
config={"configurable": {"thread_id": thread_id}}

In [36]:
input_message = {
    "role": "user",
    "content": (
        "Analyze ./data/sales_data.csv in the current dir and generate a beautiful plot. "
        "When finished, send your analysis and the plot to Slack using the tool."
    ),
}
for step in agent.stream(
    {"messages": [input_message]},
    config,
    stream_mode="updates",
):
    for _, update in step.items():
        if update and (messages := update.get("messages")) and isinstance(messages, list):
            for message in messages:
                message.pretty_print()


================================== Ai Message ==================================

    text: 📊 **SALES DATA ANALYSIS REPORT**

**1. SUMMARY STATISTICS**
• Total Revenue: $840.00
• Total Units Sold: 33 units
• Average Revenue per Day: $168.00
• Average Units per Day: 6.60 units
• Analysis Period: August 1-5, 2025

**2. PRODUCT PERFORMANCE**
🥇 Widget A: 17 units sold, $425.00 revenue (50.6%)
🥈 Widget B: 13 units sold, $325.00 revenue (38.7%)
🥉 Widget C: 3 units sold, $90.00 revenue (10.7%)

**3. KEY INSIGHTS**
✓ Widget A is the top performer in both revenue and units sold
✓ Widget B shows strong performance on Aug 5th with 8 units sold
✓ Widget C has the lowest sales volume - may need promotional support
✓ Average price per unit: Widget A ($25.00), Widget B ($25.00), Widget C ($30.00)

**4. DAILY BREAKDOWN**
• Aug 1: Widget A - 10 units, $250
• Aug 2: Widget B - 5 units, $125
• Aug 3: Widget A - 7 units, $175
• Aug 4: Widget C - 3 units, $90
• Aug 5: Widget B - 8 units, $200

📈 See attac